## Computational Theory Assessment

In [22]:
import numpy as np

## Problem 1: Binary Words and Operations

##### Implement the following functions in Python. Use numpy to ensure that all variables and values are treated as 32-bit integers. These functions are defined in the Secure Hash Standard (see page 10).
##### Document each function with a clear docstring, explain its purpose and behaviour in Markdown, and test it with appropriate examples to verify correctness.

In [23]:
def unsigned_32(x):
    return np.uint32(x);

def Parity(x, y, z):
    return unsigned_32(x) ^ unsigned_32(y) ^ unsigned_32(z)

def Ch(x, y, z):
    return unsigned_32((unsigned_32(x) & unsigned_32(y)) ^ (~unsigned_32(x) & unsigned_32(z)))

def Maj(x, y, z):
    return unsigned_32((unsigned_32(x) & unsigned_32(y)) ^ (unsigned_32(x) & unsigned_32(z)) ^ (unsigned_32(y) & unsigned_32(z)))


def rotr(x, n):
    x = unsigned_32(x)
    return unsigned_32((x >> n) | (x << unsigned_32(32 - n)))

def Sigma0(x):
    return rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22)

def Sigma1(x):
    return unsigned_32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))

def sigma0(x):
    x = unsigned_32(x)
    return unsigned_32(rotr(x, 7) ^ rotr(x, 18) ^ (x >> 3))

def sigma1(x):
    x = unsigned_32(x)
    return unsigned_32(rotr(x, 17) ^ rotr(x, 19) ^ (x >> 10))

## Problem 2: Fractional Parts of Cube Roots

In [ ]:
def primes(n):
    primes = []
    num = 2
    while len(primes) < n:
        for i in range(2,num):
            if num % i == 0:
                break
        else:
            primes.append(num)
        num += 1
    return primes
def cube_root(primes):
    cube_roots = [] 
    cube_roots = np.cbrt(primes)
    return cube_roots

def frac_32():
    p = primes(64)
    cube_roots = cube_root(p)
    frac, _= np.modf(cube_roots)
    constants = np.floor(frac * (2**32)).astype(np.uint32)
    return constants 

##primes
prime_nums = primes(64)
print(prime_nums)
print(len(prime_nums))

##cubes
cubes = cube_root(prime_nums)
print(cubes)

## fracs 
frac, nonfrac= np.modf(cubes)
print("fractional:", frac)
print("non fractional:", nonfrac)


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]
64
[1.25992105 1.44224957 1.70997595 1.91293118 2.22398009 2.35133469
 2.57128159 2.66840165 2.84386698 3.07231683 3.14138065 3.33222185
 3.44821724 3.50339806 3.60882608 3.75628575 3.89299642 3.93649718
 4.0615481  4.14081775 4.1793392  4.29084043 4.36207067 4.4647451
 4.59470089 4.65700951 4.68754815 4.7474594  4.77685618 4.83458813
 5.0265257  5.07875308 5.15513674 5.18010147 5.30145919 5.32507402
 5.39469071 5.46255557 5.50687845 5.57205466 5.63574079 5.65665283
 5.75896522 5.77899657 5.81864787 5.83827246 5.95334181 6.06412699
 6.1001702  6.11803317 6.15344949 6.20582179 6.22308425 6.30799355
 6.35786118 6.40695858 6.45531481 6.47127363 6.51868392 6.54991162
 6.56541443 6.6418522  6.74599671 6.

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords